In [ ]:
import pandas as pd
import pathlib

EMOJI_MAPPING = {
    "❤": 0,
    "😍": 1,
    "😂": 2,
    "💕": 3,
    "🔥": 4,
    "😊": 5,
    "😎": 6,
    "✨": 7,
    "💙": 8,
    "😘": 9,
    "📷": 10,
    "🇺🇸": 11,
    "☀": 12,
    "💜": 13,
    "😉": 14,
    "💯": 15,
    "😁": 16,
    "🎄": 17,
    "📸": 18,
    "😜": 19
}

print (EMOJI_MAPPING)

{'❤': 0, '😍': 1, '😂': 2, '💕': 3, '🔥': 4, '😊': 5, '😎': 6, '✨': 7, '💙': 8, '😘': 9, '📷': 10, '🇺🇸': 11, '☀': 12, '💜': 13, '😉': 14, '💯': 15, '😁': 16, '🎄': 17, '📸': 18, '😜': 19}


In [ ]:
random_examples_per_label = {}

labels = pd.read_csv("test_labels.txt", header=None)
labels.columns = ["label"]

text = pd.read_csv("test_text.txt", header=None, sep="delimiter", engine="python")
text.columns = ["text"]

text["label"] = labels["label"]

# Split the DataFrame
selected = text.iloc[:1666]          # First 1666 rows
not_selected = text.iloc[1666:]      # Remaining rows after the first 1666

# Create a dictionary to hold random samples for each label
random_examples_per_label = {}

# Group by label and sample 10 random examples for each label in not_selected
for label, group in not_selected.groupby('label'):
    random_examples_per_label[label] = group.sample(n=10, random_state=1) if len(group) >= 10 else group

#print (random_examples_per_label)

def get_query(row, shots):
  tweet=row['text']
  pattern = f"Assume that you have the following emoji list with the provided mappings {EMOJI_MAPPING}. You know that a certain person selected the following emoji as suiting a given tweet the best:\n\n"
  for lab in range(20):
    for ex in range(shots):
      pattern = pattern + 'tweet:'+random_examples_per_label[lab].iloc[ex]['text'] + ' annotation:'+str(random_examples_per_label[lab].iloc[ex]['label'])+'\n\n'
  pattern = pattern + "\nAssuming that you have to select an emoji from the given list, which suits a tweet the best, which three would you choose? Answer with one int only according to the above mappings and order chosen emojis from best suited to the worst one. Do not provide additional explanation, only the list. \n\n"+f"Tweet: {tweet}."
  return pattern

def prepare_df():

    for shots in range(2,11,2):
      column_name = f'prompt{shots}'
      selected[column_name] = selected.apply(get_query, axis=1, args=(shots,))
      #print (selected[column_name])

    return text

In [ ]:
df = prepare_df().reset_index().rename(columns={"index": "id"})


<ipython-input-7-4fc93359cd58>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected[column_name] = selected.apply(get_query, axis=1, args=(shots,))
<ipython-input-7-4fc93359cd58>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected[column_name] = selected.apply(get_query, axis=1, args=(shots,))
<ipython-input-7-4fc93359cd58>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [ ]:
# List of columns to include in the CSV file
columns_to_include = ['text', 'label']

# Add the dynamically generated prompt columns
for shots in range(2, 11, 2):
    columns_to_include.append(f'prompt{shots}')

# Export the selected columns to CSV
selected.loc[:, columns_to_include].to_csv(("21_emoji_prompts_few_shots.csv"), encoding="utf-8")